In [1]:
import os
import gc
import joblib
import pandas as pd
import numpy as np
from sklearn import metrics, preprocessing
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import callbacks
from tensorflow.keras import backend as K
from tensorflow.keras import utils


In [45]:
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [81]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [107]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("../data/train_clean.csv")

In [3]:
df.head()

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation,h1n1_vaccine,seasonal_vaccine
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,no_response,no_response,0,0
1,1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe,0,1
2,2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo,0,0
3,3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,no_response,no_response,0,1
4,4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb,0,0


In [4]:
X = df.drop(columns = ["respondent_id","h1n1_vaccine","seasonal_vaccine"])

In [6]:
categorical = ['h1n1_concern', 'h1n1_knowledge', 'behavioral_antiviral_meds',
       'behavioral_avoidance', 'behavioral_face_mask', 'behavioral_wash_hands',
       'behavioral_large_gatherings', 'behavioral_outside_home',
       'behavioral_touch_face', 'doctor_recc_h1n1', 'doctor_recc_seasonal',
       'chronic_med_condition', 'child_under_6_months', 'health_worker',
       'health_insurance', 'opinion_h1n1_vacc_effective', 'opinion_h1n1_risk',
       'opinion_h1n1_sick_from_vacc', 'opinion_seas_vacc_effective',
       'opinion_seas_risk', 'opinion_seas_sick_from_vacc', 'age_group',
       'education', 'race', 'sex', 'income_poverty', 'marital_status',
       'rent_or_own', 'employment_status', 'hhs_geo_region', 'census_msa', 'employment_industry',
       'employment_occupation']

In [8]:
#Reset the categories as a string. 
for feature in categorical: 
    X.loc[:,feature] = df[feature].astype(str)

In [9]:
for feature in categorical: 
    label_enc = preprocessing.LabelEncoder()
    X.loc[:,feature] = label_enc.fit_transform(X[feature].values)

In [12]:
y_h1n1 = df['h1n1_vaccine']
y_szn = df['seasonal_vaccine']

### Seasonal Vaccine 

In [121]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_szn, test_size = 0.2, random_state = 0)

In [122]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [123]:
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, recall_score, precision_score, accuracy_score

In [124]:
model = Sequential()

In [129]:
#The neural network model 
model.add(Dense(10, activation = 'relu', input_shape = (35,)))
model.add(Dropout(.5))
model.add(Dense(60, activation = 'relu', input_shape = (35,)))
model.add(Dropout(.5))
model.add(Dense(1, activation = 'sigmoid'))

In [130]:
model.compile(loss = 'binary_crossentropy', optimizer = 'adam',metrics=['accuracy',"AUC"] )

In [131]:
results = model.fit(X_train, y_train,
                   batch_size = 32,
                   epochs = 100,
                   validation_data = (X_test, y_test),
                   )

Epoch 1/100
668/668 [==============================] - 2s 1ms/step - loss: 0.5966 - accuracy: 0.7172 - auc: 0.7495 - val_loss: 0.5291 - val_accuracy: 0.7703 - val_auc: 0.8434
Epoch 2/100
668/668 [==============================] - 1s 1ms/step - loss: 0.5614 - accuracy: 0.7410 - auc: 0.7739 - val_loss: 0.5325 - val_accuracy: 0.7696 - val_auc: 0.8425
Epoch 3/100
668/668 [==============================] - 1s 1ms/step - loss: 0.5578 - accuracy: 0.7433 - auc: 0.7764 - val_loss: 0.5400 - val_accuracy: 0.7677 - val_auc: 0.8366
Epoch 4/100
668/668 [==============================] - 1s 1ms/step - loss: 0.5567 - accuracy: 0.7411 - auc: 0.7799 - val_loss: 0.5450 - val_accuracy: 0.7649 - val_auc: 0.8376
Epoch 5/100
668/668 [==============================] - 1s 1ms/step - loss: 0.5562 - accuracy: 0.7407 - auc: 0.7786 - val_loss: 0.5409 - val_accuracy: 0.7662 - val_auc: 0.8400
Epoch 6/100
668/668 [==============================] - 1s 1ms/step - loss: 0.5530 - accuracy: 0.7426 - auc: 0.7805 - val_loss

668/668 [==============================] - 1s 1ms/step - loss: 0.5500 - accuracy: 0.7408 - auc: 0.7823 - val_loss: 0.5505 - val_accuracy: 0.7638 - val_auc: 0.8253
Epoch 48/100
668/668 [==============================] - 1s 1ms/step - loss: 0.5503 - accuracy: 0.7415 - auc: 0.7819 - val_loss: 0.5501 - val_accuracy: 0.7677 - val_auc: 0.8226
Epoch 49/100
668/668 [==============================] - 1s 1ms/step - loss: 0.5437 - accuracy: 0.7452 - auc: 0.7884 - val_loss: 0.5470 - val_accuracy: 0.7634 - val_auc: 0.8281
Epoch 50/100
668/668 [==============================] - 1s 1ms/step - loss: 0.5459 - accuracy: 0.7434 - auc: 0.7882 - val_loss: 0.5466 - val_accuracy: 0.7660 - val_auc: 0.8265
Epoch 51/100
668/668 [==============================] - 1s 1ms/step - loss: 0.5487 - accuracy: 0.7418 - auc: 0.7841 - val_loss: 0.5476 - val_accuracy: 0.7626 - val_auc: 0.8242
Epoch 52/100
668/668 [==============================] - 1s 1ms/step - loss: 0.5477 - accuracy: 0.7417 - auc: 0.7858 - val_loss: 0.543

668/668 [==============================] - 1s 1ms/step - loss: 0.5472 - accuracy: 0.7406 - auc: 0.7856 - val_loss: 0.5563 - val_accuracy: 0.7600 - val_auc: 0.8222
Epoch 94/100
668/668 [==============================] - 1s 1ms/step - loss: 0.5458 - accuracy: 0.7429 - auc: 0.7867 - val_loss: 0.5534 - val_accuracy: 0.7598 - val_auc: 0.8216
Epoch 95/100
668/668 [==============================] - 1s 1ms/step - loss: 0.5439 - accuracy: 0.7437 - auc: 0.7907 - val_loss: 0.5478 - val_accuracy: 0.7669 - val_auc: 0.8252
Epoch 96/100
668/668 [==============================] - 1s 1ms/step - loss: 0.5482 - accuracy: 0.7387 - auc: 0.7858 - val_loss: 0.5519 - val_accuracy: 0.7602 - val_auc: 0.8209
Epoch 97/100
668/668 [==============================] - 1s 1ms/step - loss: 0.5445 - accuracy: 0.7404 - auc: 0.7896 - val_loss: 0.5568 - val_accuracy: 0.7553 - val_auc: 0.8214
Epoch 98/100
668/668 [==============================] - 1s 1ms/step - loss: 0.5440 - accuracy: 0.7427 - auc: 0.7896 - val_loss: 0.554

In [128]:
from sklearn.metrics import roc_auc_score
print(f"The accuracy for training set is {accuracy_score(y_train, model.predict_classes(X_train))}")
print(f"The AUC for training set is {roc_auc_score(y_train, model.predict_classes(X_train))}")
print(f"The accuracy for test set is {accuracy_score(y_test, model.predict_classes(X_test))}")
print(f"The AUC for test set is {roc_auc_score(y_test, model.predict_classes(X_test))}")


The accuracy for training set is 0.7780014041656915
The AUC for training set is 0.7748024604882235
The accuracy for test set is 0.7699363534256832
The AUC for test set is 0.7667298214539234


### H1N1 Vaccine 

In [112]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_h1n1, test_size = 0.2, random_state = 0)

In [113]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [114]:
results = model.fit(X_train, y_train,
                   batch_size = 32,
                   epochs = 100,
                   validation_data = (X_test, y_test),
                   )

Epoch 1/100
668/668 [==============================] - 1s 902us/step - loss: 0.4880 - accuracy: 0.7774 - auc: 0.7026 - val_loss: 0.4708 - val_accuracy: 0.7810 - val_auc: 0.7618
Epoch 2/100
668/668 [==============================] - 1s 859us/step - loss: 0.4552 - accuracy: 0.8012 - auc: 0.7373 - val_loss: 0.4601 - val_accuracy: 0.7845 - val_auc: 0.7771
Epoch 3/100
668/668 [==============================] - 1s 863us/step - loss: 0.4400 - accuracy: 0.8098 - auc: 0.7556 - val_loss: 0.4510 - val_accuracy: 0.7935 - val_auc: 0.7899
Epoch 4/100
668/668 [==============================] - 1s 863us/step - loss: 0.4287 - accuracy: 0.8220 - auc: 0.7691 - val_loss: 0.4362 - val_accuracy: 0.8104 - val_auc: 0.7998
Epoch 5/100
668/668 [==============================] - 1s 876us/step - loss: 0.4152 - accuracy: 0.8264 - auc: 0.7860 - val_loss: 0.4212 - val_accuracy: 0.8162 - val_auc: 0.8109
Epoch 6/100
668/668 [==============================] - 1s 862us/step - loss: 0.4128 - accuracy: 0.8283 - auc: 0.789

668/668 [==============================] - 1s 864us/step - loss: 0.3753 - accuracy: 0.8428 - auc: 0.8390 - val_loss: 0.3949 - val_accuracy: 0.8308 - val_auc: 0.8501
Epoch 48/100
668/668 [==============================] - 1s 860us/step - loss: 0.3751 - accuracy: 0.8432 - auc: 0.8386 - val_loss: 0.3914 - val_accuracy: 0.8328 - val_auc: 0.8512
Epoch 49/100
668/668 [==============================] - 1s 881us/step - loss: 0.3774 - accuracy: 0.8414 - auc: 0.8381 - val_loss: 0.3949 - val_accuracy: 0.8340 - val_auc: 0.8505
Epoch 50/100
668/668 [==============================] - 1s 849us/step - loss: 0.3758 - accuracy: 0.8403 - auc: 0.8378 - val_loss: 0.3935 - val_accuracy: 0.8351 - val_auc: 0.8494
Epoch 51/100
668/668 [==============================] - 1s 847us/step - loss: 0.3773 - accuracy: 0.8401 - auc: 0.8369 - val_loss: 0.3956 - val_accuracy: 0.8257 - val_auc: 0.8503
Epoch 52/100
668/668 [==============================] - 1s 850us/step - loss: 0.3743 - accuracy: 0.8412 - auc: 0.8396 - val

668/668 [==============================] - 1s 937us/step - loss: 0.3754 - accuracy: 0.8395 - auc: 0.8406 - val_loss: 0.3931 - val_accuracy: 0.8302 - val_auc: 0.8502
Epoch 94/100
668/668 [==============================] - 1s 917us/step - loss: 0.3736 - accuracy: 0.8421 - auc: 0.8418 - val_loss: 0.3906 - val_accuracy: 0.8334 - val_auc: 0.8505
Epoch 95/100
668/668 [==============================] - 1s 905us/step - loss: 0.3728 - accuracy: 0.8412 - auc: 0.8415 - val_loss: 0.3971 - val_accuracy: 0.8315 - val_auc: 0.8503
Epoch 96/100
668/668 [==============================] - 1s 885us/step - loss: 0.3752 - accuracy: 0.8432 - auc: 0.8390 - val_loss: 0.3933 - val_accuracy: 0.8343 - val_auc: 0.8510
Epoch 97/100
668/668 [==============================] - 1s 951us/step - loss: 0.3762 - accuracy: 0.8417 - auc: 0.8387 - val_loss: 0.3948 - val_accuracy: 0.8253 - val_auc: 0.8506
Epoch 98/100
668/668 [==============================] - 1s 869us/step - loss: 0.3746 - accuracy: 0.8419 - auc: 0.8399 - val

In [120]:

print(f"The accuracy for training set is {accuracy_score(y_train, model.predict_classes(X_train))}")
print(f"The AUC for training set is {roc_auc_score(y_train, model.predict_classes(X_train))}")
print(f"The accuracy for test set is {accuracy_score(y_test, model.predict_classes(X_test))}")
print(f"The AUC for test set is {roc_auc_score(y_test, model.predict_classes(X_test))}")


The accuracy for training set is 0.8192838754973086
The AUC for training set is 0.5784591582222335
The accuracy for test set is 0.806626731561213
The AUC for test set is 0.5686946349698027
